# Programme de Transposition des notes Moodle vers SynapseS


The program works with <span style="color:red">the original</span> Moodle and SynapseS file formats. If you make changes to the content, <span style="color:red">you may not change the order and name of the columns</span>.
> 
>1. Download the **Excel** (from Moodle) and **CSV** (from SynapseS) files by clicking on the corresponding buttons
>2. Transpose the lettered notes to SynapseS by clicking on the **Process** button. <br>
    _the decimal separator for notes can be either , (comma) or . (dot) depending on the needs of SynapseS_
>3. Retrieve the generated csv file by clicking on the **'Download xxx.csv'** button that will appear.
> 
_Version 2.0_

In [ ]:
import ipywidgets as ipw
from ipywidgets import FileUpload
from IPython.display import display, HTML, clear_output
from IPython.display import FileLink, FileLinks

from ipywidgets import Output, Button
from base64 import b64encode

import pandas as pd
import io,os

def LetterToNum(letter,mode='TP'):
    if mode == 'X':
        # -- X --    
        if letter == 'A+':
            note = 19
        elif letter == 'A':
            note = 17
        elif letter == 'A-':
            note = 15
        elif letter == 'B+':
            note = 14
        elif letter == 'B':
            note = 13
        elif letter == 'B-':
            note = 12
        elif letter == 'C':
            note = 11
        elif letter == 'D':
            note = 9
        elif letter == 'E':
            note = 7
        elif letter == 'F':
            note = 0
        else:
            note = None
    else:
        # -- Telecom --
        if letter == 'A+':
            note = 17.1
        elif letter == 'A':
            note = 16.9
        elif letter == 'A-':
            note = 13.9
        elif letter == 'B+':
            note = 13.9
        elif letter == 'B':
            note = 12.9
        elif letter == 'B-':
            note = 10.9
        elif letter == 'C':
            note = 9.9
        elif letter == 'D':
            note = 7.9
        elif letter == 'E':
            note = 5
        elif letter == 'F':
            note = 0
        else:
            note = None
    return note

def on_upload_change(change):
    #print(change)
    if len(upload1.value)>0 and len(upload2.value)>0:
        button.disabled=False
    
def download_data(e=None):
    #ici lire le fichier csv à envoyer en premier parametre
    with open(cours, 'rb') as f:
        data = f.read().decode('utf-8-sig')
    trigger_download(data, cours, kind='text/csv')

def trigger_download(text, filename, kind='text/json'):
    # see https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/Data_URIs for details
    content_b64 = b64encode(text.encode()).decode()
    data_url = f'data:{kind};charset=utf-8;base64,{content_b64}'
    js_code = f"""
        var a = document.createElement('a');
        a.setAttribute('download', '{filename}');
        a.setAttribute('href', '{data_url}');
        a.click()
    """
    with out:
        clear_output()
        upload1.value.clear()
        upload1._counter = 0
        upload2.value.clear()        
        upload2._counter = 0
        output1.value = ''
        output2.value = ''
        output3.value = ''
        output4.value = ''
        display(HTML(f'<script>{js_code}</script>'))
    
def on_button_clicked(a):
    global cours
    with out:
        clear_output()
        
    if upload1.value[list(upload1.value)[0]]['metadata']['type'] != 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet':
        upload1.value.clear()
        upload1._counter = 0
        with out:
            display(ipw.Textarea(
            value="Moodle file error, must be an xls\nThe current type is: "+upload1.value[list(upload1.value)[0]]['metadata']['type'],
            description='Error:',
            disabled=False,
            layout=ipw.Layout(width="50%")))            
        return
    
    if upload2.value[list(upload2.value)[0]]['metadata']['type'] != 'text/csv':
        upload2.value.clear()        
        upload2._counter = 0        
        with out:
            display(ipw.Textarea(
            value="The SynapseS file does not have the expected format, must be an xls\nThe current type is: " + upload2.value[list(upload2.value)[0]]['metadata']['type'],
            description='Error:',
            disabled=False,
            layout=ipw.Layout(width="50%")))
        return

    moodle_file = list(upload1.value.values())[0]
    content1 = moodle_file['content']
    content1 = io.BytesIO(content1)
    dfMoodle = pd.read_excel(content1.read()) 
    output1.value = dfMoodle.head().style.hide_index().to_html() 
    
    synapses_file = list(upload2.value.values())[0]
    content2 = synapses_file['content']
    content2 = io.StringIO(content2.decode('utf-8'))
    dfSynapses = pd.read_csv(content2, delimiter=';', encoding='utf-8-sig',skiprows=1, on_bad_lines='skip')
    output2.value = dfSynapses.head().style.hide_index().to_html()
    
    try:
        s1 = dfSynapses['Matricule']
    except Exception as e:
        upload2.value.clear()        
        upload2._counter = 0        
        with out:
            display(ipw.Textarea(
                value = "The 'Matricule' column does not appear in the SynapseS file.\nEnd of processing.\n",
                description = "Error:",
                disable=False,
                layout=ipw.Layout(width="100%")))
        return

    try:
        s2 = dfMoodle['Institution']
    except Exception as e:
        upload1.value.clear()        
        upload1._counter = 0
        with out:
            display(ipw.Textarea(
                value = "The 'Institution' column does not appear in the Moodle file.\nEnd of processing.",
                description = "Error:",
                disable=False,
                layout=ipw.Layout(width="100%")))  
            
    intersection = pd.Series(list(set(s1) & set(s2)),dtype='string')

    try:
        #if dfMoodle[['Institution','First name','Surname','Email address']][~dfMoodle['Institution'].isin(intersection)].shape[0] >0:
        if dfMoodle.iloc[:, [3,0,1,5]][~dfMoodle['Institution'].isin(intersection)].shape[0] >0:
            #output3.value = "<b>Les éléments suivants n'étaient pas dans SynapseS :</b>" + dfMoodle[['Institution','First name','Surname','Email address']][~dfMoodle['Institution'].isin(intersection)].style.hide_index().to_html()
            output3.value = "<b>The following items were not in SynapseS:</b>" + dfMoodle.iloc[:, [3,0,1,5]][~dfMoodle['Institution'].isin(intersection)].style.hide_index().to_html()
            accordion.set_title(2,'⚠️ Comparison 🚨')
    except Exception as e:
        with out:
            display(ipw.Textarea(
                value=e,
                description = 'Error:',
                disable=False,
                layout=ipw.Layout(width="100%")))
        return

    # on récupère le nom de l'avant-dernière colonne
    colname = dfMoodle.columns[len(dfMoodle.columns) -2 ]
    if not (('lettre' not in colname.lower()) or ('letter' not in colname.lower())):
        upload1.value.clear()        
        upload1._counter = 0
        with out:
            display(ipw.Textarea(
            value= "The Moodle file does not appear to have lettered grades. Do an export with the 'Letter' option enabled.",
            description='Error:',
            disabled=False,
            layout=ipw.Layout(width="500px")))
        return
        
    for e in intersection:
        dfSynapses.at[dfSynapses['Matricule'] == e,'Note initiale'] = LetterToNum(dfMoodle.loc[dfMoodle['Institution'] == e][colname].values[0])

    # on écrit le csv
    if cBox.value == True:
        decimal = ','
    else:
        decimal = '.'
    dfSynapses.to_csv('out.csv',sep=';', decimal=decimal, encoding='utf-8', index=False)

    # on lit le fichier original pour récupérer la première ligne et la première colonne
    # On récupère le nom du cours
    content2 = synapses_file['content']
    cours = content2.decode('utf-8-sig').split("\r\n",1)[0].split(';')[0]
    
    # On lit le fichier csv temporaire
    with open("out.csv",'r') as csv:
        csvlines = []
        for line in csv:
            csvlines.append(line)

    # On fusionne les informations
    with open (f'{cours}.csv', 'w', encoding='utf-8-sig') as fp:
        fp.write(content2.decode('utf-8-sig').split("\r\n",1)[0]+"\r\n")
        for l in csvlines:
            fp.write(l)

    # on efface le fichier temporaire
    os.remove('out.csv')

    # on affiche le résultat
    #display(HTML(f'<br>Le fichier resultat à télécharger est : {cours}.csv'),FileLink(f'{cours}.csv'))
    output4.value = f'The result file to download is: {cours}.csv<br>' 
    # modifier titre du bouton
    cours = f'{cours}.csv'
    btn.description = f"Download {cours}"
    with out:
        #display(FileLink(cours))
        display(btn)

cours = ''
upload1 = FileUpload(accept='*.xlxs', multiple=False, description='Moodle (.xlsx)', layout=ipw.Layout(width="200px"))
upload2 = FileUpload(accept='*.csv', multiple=False, description='SynapseS (.csv)', layout=ipw.Layout(width="200px"))
button = ipw.Button(description="Traitement", layout=ipw.Layout(width="200px"),  disabled=True)
cBox = ipw.Checkbox(value=True,description="Decimal separator ' , '",disabled=False,indent=False)

display(ipw.HBox([upload1,upload2,button,cBox])) 
button.on_click(on_button_clicked)
upload1.observe(on_upload_change, names='_counter')
upload2.observe(on_upload_change, names='_counter')

out = ipw.Output()
btn = Button(description=f'Download {cours}', layout=ipw.Layout(width="600px"))
btn.on_click(download_data)
output1 = ipw.HTML(value="", placeholder="", layout=ipw.Layout(width="100%", overflow_y='auto'),  disabled=True)
output2 = ipw.HTML(value="", placeholder="", layout=ipw.Layout(width="100%", overflow_y='auto'),  disabled=True)
output3 = ipw.HTML(value="", placeholder="", layout=ipw.Layout(width="100%", overflow_y='auto'),  disabled=True)
output4 = ipw.HTML(value="", placeholder="", layout=ipw.Layout(width="100%", overflow_y='auto'),  disabled=True)
accordion = ipw.Accordion(children=[output1,output2,output3,output4], selected_index = None )
accordion.set_title(0,'Moodle')
accordion.set_title(1,'Synapse')
accordion.set_title(2,'Comparison')
accordion.set_title(3,'Output')

display(out,accordion)